<a href="https://colab.research.google.com/github/RussAbbott/Wyrdl/blob/main/Wyrdl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install "rich[jupyter]"


# -------------------------------------- wyrdl.py --------------------------------------
#            Adapted from: https://realpython.com/python-wordle-clone/
# --------------------------------------------------------------------------------------


import contextlib
import pathlib
import random
from string import ascii_letters, ascii_uppercase
from typing import Dict, Iterable, List, Literal, Sequence, Tuple


class Rich_IO():

    from rich.console import Console
    from rich.theme import Theme

    WYRDL_THEME = Theme({'correct':   'bold black on #ddffdd',
                         'incorrect': 'bold black on #ffdddd',
                         'warning':   'bold red on #eeffee',
                         })

    console = Console(width=80, theme=WYRDL_THEME, highlight=False)

    @staticmethod
    def guess_a_word(previous_guesses) -> str:
        guess = Rich_IO.console.input("\nGuess word: ").upper()

        if guess in previous_guesses:
            Rich_IO.rich_print(f"You've already guessed {guess}.", style="warning")
            return Rich_IO.guess_a_word(previous_guesses)

        if len(guess) != NUM_LETTERS:
            Rich_IO.rich_print(
                f"Your guess must be {NUM_LETTERS} letters.", style="warning"
            )
            return Rich_IO.guess_a_word(previous_guesses)

        if any((invalid := letter) not in ascii_letters for letter in guess):
            Rich_IO.rich_print(
                f"Invalid letter: '{invalid}'. Please use English letters.", style="warning",
            )
            return Rich_IO.guess_a_word(previous_guesses)

        return guess

    @staticmethod
    def rich_print(x, style=None, justify=None, end='\n', sep=' '):
        Rich_IO.console.print(x, style=style, justify=justify)

    @staticmethod
    def show_guesses_and_letters(guesses: List[str], guess_scores: List[str], letter_to_score_map: Dict[str, str]):

        # Print the sequences of guesses, one styled guess per line.
        for guess, guess_score in zip(guesses, guess_scores):
            # guess_score is a string of letter scores.
            Rich_IO.rich_print(Rich_Styles.join_with_scores(guess, guess_score, ' '), justify="center")

        # Clever unzip()
        letters, scores = zip(*letter_to_score_map.items())

        # scores is of type Tuple[str, str, ..., str] with an unknown number of elements. (Each element is a
        # single character.) We can pass it to scores_to_styles() by declaring the scores_to_styles() parameter
        # as an Iterable or a Sequence--or by not declaring the scores_to_styles() parameter at all!
        Rich_IO.rich_print(f"\n{Rich_Styles.join_with_scores(letters, scores, ' ')} \n\n", justify="center")

    @staticmethod
    def refresh_page(headline):
        Rich_IO.console.clear()
        Rich_IO.console.rule(f"[#00CCFF]:mage: {headline} :bulb:[/]\n")


class Rich_Styles():

    @staticmethod
    def add_style(elt, style):
        return f"[{style}]{elt}[/]"

    # STYLES maps scores to styles. E.g., the score 'g' (for 'green') is mapped to the style
    # "bold black on #00FF00". The score '_', which means no information (yet) about this letter,
    # is mapped to 'dim'. Wouldn't help to add these to the Theme since a Theme style can only be
    # applied to a complete print statement, not to a segment.
    STYLES = {'g': "bold black on #00ff00",
              'y': "bold black on #ffff00",
              '~': "bold white on #000000",
              '_': "dim",
              }

    @staticmethod
    def join_with_scores(letters, scores, join_elt) -> str:
        styles = Rich_Styles.scores_to_styles(scores)
        return Rich_Styles.join_with_styles(letters, styles, join_elt)

    @staticmethod
    def join_with_styles(letters, styles, join_elt) -> str:
        styled_letters = [Rich_Styles.add_style(letter, letter_style) for letter, letter_style in zip(letters, styles)]
        return join_elt.join(styled_letters)

    @staticmethod
    def scores_to_styles(scores: Iterable[Literal['g', 'y', '~', '_']]) -> List[str]:
        # The declaration of scores is not necessary. But if we want to declare it, we must declare
        # it as an Iterable or Sequence. (Sequence is a subtype of Iterable that allows access by
        # index.) This allows us to pass to scores_to_styles() arguments of both type str and type
        # Tuple[str, str, ..., str], i.e., a tuple of an unknown number of strings. We pass both
        # from show_guesses_and_letters(). See above in Rich_IO.
        return [Rich_Styles.STYLES[score] for score in scores]




In [2]:

NUM_LETTERS = 5
NUM_GUESSES = 6

# SCORE_RANKING assigns an integer to each score so that they can be compared with '>'.
# The ordering is '_~yg'.
SCORE_RANKING: Dict[str, int] = {score: rank for rank, score in enumerate('_~yg')}

verbose: bool

def main(verbo=False):
    global verbose
    verbose = verbo

    # ------- Pre-process: get the secret word and set up lists and a dictionary -------

    # secret_word = get_random_word(WORDS_PATH.read_text(encoding="utf-8").split("\n"))
    secret_word = random.choice(WYRLD_WORDS)
    if verbose:
        print(secret_word)

    # letter_to_score_map is a dictionary of letters mapped to scores.
    # The scores are updated as new information is found. Initially, we
    # have no information about any letter.
    letter_to_score_map: Dict[str, str] = {letter: '_' for letter in ascii_uppercase}

    # guesses is a list of the guesses
    guesses: List[str] = []

    # scores is a list of the guess scores. Each guess score is a string of letter scores.
    # Must keep track of the scores so that when we display the sequences of guesses,
    # earlier guesses will have the score valid at the time.
    guess_scores: List[str] = []

    # ------------------------Main loop: make the guesses------------------------------

    found_it = False
    with contextlib.suppress(KeyboardInterrupt):
        for idx in range(NUM_GUESSES):
            Rich_IO.refresh_page(headline=f"Guess {idx + 1}")

            # Make and score a guess and update guesses, scores, and letter_to_score_map
            guesses, guess_scores, letter_to_score_map = make_a_guess(secret_word, guesses, guess_scores, letter_to_score_map)
            Rich_IO.show_guesses_and_letters(guesses, guess_scores, letter_to_score_map)
            if found_it := guesses[-1] == secret_word:
                break

    # ---------------------- Post-process: report the result ----------------------------

    (correct_sorry, is_was, style) = ('Correct', 'is',  'correct') if found_it else \
                                     ('Sorry',   'was', 'incorrect')
    Rich_IO.rich_print(f"\n {correct_sorry}, the secret word {is_was} {secret_word} ", style=style)

def make_a_guess(secret_word, guesses, guess_scores, letter_to_score_map) -> Tuple[List[str], List[str], Dict[str, str]]:
    """
    Make and score a new guess. Update and return guesses, guess_scores, and letter_to_score_map.
    """
    guess = Rich_IO.guess_a_word(previous_guesses=guesses)
    guess_score = score_a_guess(secret_word, guess)

    # Update letter_to_score_map
    for letter, letter_score in zip(guess, guess_score):
        if SCORE_RANKING[letter_score] > SCORE_RANKING[letter_to_score_map[letter]]:
            letter_to_score_map[letter] = letter_score

    # Use + instead of append() to be more functional.
    # When using +, must put guess into a list since + concatenates its arguments.
    return guesses + [guess], guess_scores + [guess_score], letter_to_score_map

def score_a_guess(secret_word, guess) -> str:
    '''
    Build a list with one char for each guess element.
    g = Letter in Correct position (green);
    y = Letter in Incorrect position (yellow);
    ~ = Letter not in secret_word. (These are the gray results. Can't use 'g' since we use it for green)

    In the following example: secret_word = 'hpaay'
                              guess       = 'aaapz'
                              -> score    = 'y~gy~'

    The third letter in both the secret_word and the guess is "a".
        So the third letter in the score must be "g".
        (The matching letters must be determined first.)
    The first letter in the guess, 'a', is associated with the fourth letter in the secret_word, 'a'.
        So the first letter in the score is "y".
    The second letter in the guess, 'a', has nothing in secret_word to be associated with.
        So the second letter in the score is "#".
    The other two letter in the score should be self explanatory.
    '''

    # Generate the score list with the letters in the correct position marked 'g'.
    # All non-green elements are marked '#'. These will be changed to 'y' later if appropriate.
    score = [('g' if sec_ltr == guess_ltr else '~') for sec_ltr, guess_ltr in zip(secret_word, guess)]

    # Generate a list of the "holes," i.e., secret_word letters that do not have correct corresponding guess letters.
    secret_word_holes = [sec_ltr for sec_ltr, score_ltr in zip(secret_word, score) if score_ltr != 'g']

    if verbose: print(f'{secret_word_holes = }')

    # For each available secret_word hole find matching guess letters. Ignore guess letters that are
    # paired with their secret_word letters. Score the first match as 'y' and don't look further.
    for ltr in secret_word_holes:
        for i in range(len(score)):
            # Have we found a hole-filling letter in guess that is not otherwise committed?
            # If so, mark it 'y'. Since we have now filled this hole, go on to the next hole.
            if guess[i] == ltr and score[i] == '~':
                score[i] = 'y'
                break

    # Return score as a string
    score_string = ''.join(score)
    if verbose:
        print(f'\n\t{score_string = }\n\n')
    return score_string

# main()

In [3]:
# This is the start of a longer word list.
# It's the final cell to make it easier to consult while playing.

wordle_word_list = [
    'aback', 'abase', 'abate', 'abbey', 'abbot', 'abhor', 'abide', 'abled', 'abode', 'abort',
    'about', 'above', 'abuse', 'abyss', 'acorn', 'acrid', 'actor', 'acute', 'adage', 'adapt',
    'adept', 'adieu', 'admin', 'admit', 'adobe', 'adopt', 'adore', 'adorn', 'adult', 'affix',
    'afire', 'afoot', 'afoul', 'after', 'again', 'agape', 'agate', 'agent', 'agile', 'aging',
    'aglow', 'agony', 'agora', 'agree', 'ahead', 'aider', 'aisle', 'alarm', 'album', 'alert',
    'algae', 'alibi', 'alien', 'align', 'alike', 'alive', 'allay', 'alley', 'allot', 'allow',
    'alloy', 'aloft', 'alone', 'along', 'aloof', 'aloud', 'alpha', 'altar', 'alter', 'amass',
    'amaze', 'amber', 'amble', 'amend', 'amiss', 'amity', 'among', 'ample', 'amply', 'amuse',
    'angel', 'anger', 'angle', 'angry', 'angst', 'anime', 'ankle', 'annex', 'annoy', 'annul',
    'anode', 'antic', 'anvil', 'aorta', 'apart', 'aphid', 'aping', 'apnea', 'apple', 'apply',
    'apron', 'aptly', 'arbor', 'ardor', 'arena', 'argue', 'arise', 'armor', 'aroma', 'arose',
    'array', 'arrow', 'arson', 'artsy', 'ascot', 'ashen', 'aside', 'askew', 'assay', 'asset',
    'atoll', 'atone', 'attic', 'audio', 'audit', 'augur', 'aunty', 'avail', 'avert', 'avian',
    'avoid', 'await', 'awake', 'award', 'aware', 'awash', 'awful', 'awoke', 'axial', 'axiom',
    'axion', 'azure']

WYRLD_WORDS = [word.upper() for word in wordle_word_list
                    if len(word) == NUM_LETTERS and
                       all(letter in ascii_letters for letter in word)]

if not WYRLD_WORDS:

    Rich_IO.rich_print(
        f"No words of length {NUM_LETTERS} in the word list",
        style="warning")

    raise SystemExit()

main()

──────────────────────────────── 🧙 Guess 1 💡  ────────────────────────────────

Guess word:

ouise


O U I S E

A B C D E F G H I J K L M N O P Q R S T U V W X Y Z

...

O U I S E

A N G E L

A N G E R

A B C D E F G H I J K L M N O P Q R S T U V W X Y Z

 Correct, the secret word is ANGER 